In [1]:
import pandas as pd
df_aug = pd.read_csv('data/train_clean_v2_gpt.csv')

In [9]:
df_train = pd.read_csv('data/train.csv')

In [11]:
X_train = df_train["tweet"].tolist()


In [12]:
x_gpt_data = gpt_data['tweet_aug'].to_list()

In [16]:
X_train.extend(x_gpt_data)

In [17]:
len(X_train)

10784

In [15]:
len(x_gpt_data)

3500

In [5]:
df_aug = df_aug.fillna(0)
gpt_data = df_aug[df_aug['index']==0][['index', 'clean_tweet_remove_hashtag', 'label']]

In [6]:
gpt_data

,index,clean_tweet_remove_hashtag,label
7284,0.0,"Global warming, which refers to the continuous...",2
7285,0.0,The term climate change encompasses global war...,2
7286,0.0,Climate change is a term used to describe glob...,2
7287,0.0,Global warming and its impact on Earth's clima...,2
7288,0.0,Climate change refers to the continuous rise i...,2
...,...,...,...
10779,0.0,"Global warming, which refers to the continuous...",2
10780,0.0,The term climate change encompasses global war...,2
10781,0.0,Climate change is a term used to describe glob...,2
10782,0.0,Global warming and its impact on Earth's clima...,2


In [7]:
gpt_data = gpt_data.rename(columns={'clean_tweet_remove_hashtag': 'tweet_aug'})

In [8]:
gpt_data.to_csv('data/train_gpt_aug.csv')

In [71]:
import pandas as pd
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

In [74]:
df_train = pd.read_csv('data/train.csv')[['index', 'tweet', 'label']]
df_dev = pd.read_csv('data/dev.csv')[['index', 'tweet', 'label']]
df_test = pd.read_csv('data/test.csv')[['index', 'tweet', 'label']]

In [75]:
def preprocess_tweet(tweet):
    # remove URLs
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', tweet)
    tweet = re.sub(r'http\S+', ' ', tweet)
    
    # remove usernames
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    # remove the # in hashtag and split hashtags
    tweet_toks = tweet.split(" ")
    final_tweet_toks = []
    for i in range(len(tweet_toks)):
        if tweet_toks[i].startswith("#"):
            hashtag = tweet_toks[i]
            hashtag = hashtag[1:]
            split_hashtag = re.findall('[0-9]+|[A-Z][a-z]+|[A-Z][A-Z]+|[a-z]+', hashtag)
            final_tweet_toks = final_tweet_toks + split_hashtag
        else:
            final_tweet_toks.append(tweet_toks[i])
    tweet = " ".join(final_tweet_toks)
    
    # convert text to lower-case
    tweet = tweet.lower()
    
    #Remove any other punctuation
    tweet = [char if char not in string.punctuation else ' ' for char in tweet ]
    tweet = ''.join(tweet)
    
    #Remove non-ascii characters
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)
    
    #Remove stopwords and emoticons from final word list
    tokens = word_tokenize(tweet)
    stop_words = set(stopwords.words('english'))
    filtered_tweet = []
    for w in tokens:
        if w not in stop_words:
            filtered_tweet.append(w)
            
    return ' '.join(filtered_tweet)


In [76]:
import re
def remove_url(tweet):
    # remove URLs
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', tweet)
    tweet = re.sub(r'http\S+', ' ', tweet)
    return tweet

def remove_username(tweet):
    return re.sub('@[^\s]+', ' ', tweet)

def splite_hashtag(tweet):
    # remove the # in hashtag and split hashtags
    tweet_toks = tweet.split(" ")
    final_tweet_toks = []
    for i in range(len(tweet_toks)):
        if tweet_toks[i].startswith("#"):
            hashtag = tweet_toks[i]
            hashtag = hashtag[1:]
            split_hashtag = re.findall('[0-9]+|[A-Z][a-z]+|[A-Z][A-Z]+|[a-z]+', hashtag)
            final_tweet_toks = final_tweet_toks + split_hashtag
        else:
            final_tweet_toks.append(tweet_toks[i])
    tweet = " ".join(final_tweet_toks)

    return tweet

In [77]:
def apply_preprocess(df):
    df["tweet_remove_url"] = df["tweet"].apply(remove_url)
    df["tweet_remove_username"] = df["tweet"].apply(remove_username)
    df["tweet_remove_url_username"] = df["tweet_remove_url"].apply(remove_username)
    df["tweet_remove_url_username_splite_hasghtag"] = df["tweet_remove_url_username"].apply(splite_hashtag)
    df["tweet_remove_url_username_splite_hasghtag_lower_case"] = df["tweet_remove_url_username_splite_hasghtag"].apply(lambda tweet : tweet.lower())
    df["tweet_complete_cleaning"] = df["tweet"].apply(preprocess_tweet)
    
    return df

In [78]:
df_train = apply_preprocess(df_train)
df_dev = apply_preprocess(df_dev)
df_test = apply_preprocess(df_test)

In [81]:
df_train.to_csv('data/train_clean.csv')
df_dev.to_csv('data/dev_clean.csv')
df_test.to_csv('data/test_clean.csv')

In [80]:
df_train.columns

Index(['index', 'tweet', 'label', 'tweet_remove_url', 'tweet_remove_username',
       'tweet_remove_url_username',
       'tweet_remove_url_username_splite_hasghtag',
       'tweet_remove_url_username_splite_hasghtag_lower_case',
       'tweet_complete_cleaning'],
      dtype='object')